In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
from folium.plugins import HeatMap
from sklearn.preprocessing import MinMaxScaler

In [33]:
file_path = 'C:/Users/nilsk/Dokumente/Machine Learning (MSc.)/1. Semester/Data Literacy/preprocessed_cities/berlin/city_listings.csv'
city_listings = pd.read_csv(file_path, index_col=0)

city_listings['price'] = city_listings['price'].replace('[\$,]', '', regex=True).astype(float).to_numpy()
price_prediction = city_listings['price'].mean()

scaler = MinMaxScaler()

city_listings['price_difference'] = city_listings['price'].apply(lambda x: x - price_prediction)

below_prediction = city_listings[city_listings['price_difference'] < 0].copy()
above_prediction = city_listings[city_listings['price_difference'] >= 0].copy()

below_prediction['price_difference'] = below_prediction['price_difference'] * -1

above_prediction['price_difference_norm'] = scaler.fit_transform(above_prediction[['price_difference']]) 

below_prediction['price_difference_norm'] = scaler.fit_transform(below_prediction[['price_difference']]) 



data_above_prediction = above_prediction[['latitude', 'longitude','price_difference_norm']].values.tolist()
data_below_prediction = below_prediction[['latitude', 'longitude','price_difference_norm']].values.tolist()

map_center = [city_listings['latitude'].mean(), city_listings['longitude'].mean()]

<>:4: SyntaxWarning: invalid escape sequence '\$'
<>:4: SyntaxWarning: invalid escape sequence '\$'
C:\Users\nilsk\AppData\Local\Temp\ipykernel_21864\2694236209.py:4: SyntaxWarning: invalid escape sequence '\$'
  city_listings['price'] = city_listings['price'].replace('[\$,]', '', regex=True).astype(float).to_numpy()


In [34]:
m = folium.Map( zoom_start=6, control_scale=True, location = map_center)


In [38]:
if not below_prediction.empty:
    HeatMap(
        data=data_below_prediction,
        gradient={0: 'green', 1: 'blue'},  # Solid blue for all weights
        radius=10,
        blur=15,
        max_zoom=10,
        name='Below Prediction'
    ).add_to(m)

if not above_prediction.empty:
    HeatMap(
        data=data_above_prediction,
        gradient={0: 'green', 1: 'red'},  # Solid red for all weights
        radius=10,
        blur=15,
        max_zoom=10,
        name='Above Prediction'
    ).add_to(m)

# Add layer control to toggle visibility
folium.LayerControl().add_to(m)

m